In [2]:
import numpy as np
import pandas as pd
import re

In [3]:
file_path = 'resultados_secao_absolutos/EDUARDO_DA_COSTA_PAES.csv'
data = pd.read_csv(file_path, index_col=0)

In [4]:
data.head(2)

,candidato,municipio,local,zona,secao,turno,votos_recebidos,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_ensino_fundamental_incompleto,...,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_16_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,ds_genero_não_informado,eleitores_totais,percentual_votos
0,EDUARDO DA COSTA PAES,60011,1317,245,254,1,170,90,38,60,...,8,10,0,1,0,263,194,2,459,0.370370
1,EDUARDO DA COSTA PAES,60011,1341,123,262,1,155,70,18,100,...,10,11,0,0,0,239,186,0,425,0.364706


In [8]:
remove_cols = [col for col in data.columns if re.search('não_informado', col) or re.search('inválida', col)]
data = data.drop(remove_cols, axis=1)

In [10]:
pesos_escolaridade = {
    'ds_grau_escolaridade_analfabeto': 0,
    'ds_grau_escolaridade_lê_e_escreve': 1,
    'ds_grau_escolaridade_ensino_fundamental_incompleto': 2,
    'ds_grau_escolaridade_ensino_fundamental_completo': 3,
    'ds_grau_escolaridade_ensino_médio_incompleto': 4,
    'ds_grau_escolaridade_ensino_médio_completo': 5,
    'ds_grau_escolaridade_superior_incompleto': 6,
    'ds_grau_escolaridade_superior_completo': 7
}

# Calculando o score usando np.average com os pesos
data["escolaridade_score"] = data.apply(
    lambda row: np.average(
        [row[col] for col in pesos_escolaridade.keys()],  # Valores das colunas
        weights=[pesos_escolaridade[col] for col in pesos_escolaridade.keys()]  # Pesos associados às colunas
    ),
    axis=1
)

In [11]:
data = data.drop(pesos_escolaridade.keys(), axis=1)
data.head(1)

,candidato,municipio,local,zona,secao,turno,votos_recebidos,ds_raca_cor_parda,ds_raca_cor_branca,ds_raca_cor_preta,...,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_16_anos,ds_genero_feminino,ds_genero_masculino,eleitores_totais,percentual_votos,escolaridade_score
0,EDUARDO DA COSTA PAES,60011,1317,245,254,1,170,6,15,4,...,4,8,10,0,1,263,194,459,0.37037,64.964286


In [18]:
def extrair_media_idade(col):
    try:
        return (float(col.split('_')[3]) + float(col.split('_')[5])) / 2
    except:
        try:
            return (float(col.split('_')[3]))
        except:
            return 'coluna invalida'

In [23]:
etaria_cols = [col for col in data.columns if re.search('faixa_etaria', col)]
etaria_cols_weights = {
    col: extrair_media_idade(col)
    for col in etaria_cols
}

data['idade_score'] = data.apply(
    lambda row: np.average(
        [row[col] for col in etaria_cols_weights.keys()],
        weights=[etaria_cols_weights[col] for col in etaria_cols_weights.keys()]
    ),
    axis=1
)


In [25]:
data = data.drop(etaria_cols_weights.keys(), axis=1)
data.head(1)

,candidato,municipio,local,zona,secao,turno,votos_recebidos,ds_raca_cor_parda,ds_raca_cor_branca,ds_raca_cor_preta,ds_raca_cor_amarela,ds_raca_cor_indígena,ds_genero_feminino,ds_genero_masculino,eleitores_totais,percentual_votos,escolaridade_score,idade_score
0,EDUARDO DA COSTA PAES,60011,1317,245,254,1,170,6,15,4,0,0,263,194,459,0.37037,64.964286,21.258206
